In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Data Load and Processing

In [34]:
iris_data = load_iris()
df = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
df['target'] = iris_data.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [36]:
# Split dataset into train, test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

In [37]:
# features extracts
X_train, y_train = train_df.drop(columns='target'), train_df['target']
X_test, y_test = test_df.drop(columns='target'), test_df['target']

In [38]:
# Normalized
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# Convert numpy to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

## Build Neural Network

In [40]:
class ClassificationNN(nn.Module):
  def __init__(self, input_sz):
    super(ClassificationNN, self).__init__()

    self.network = nn.Sequential(
        nn.Linear(input_sz, 16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 3)
    )

  def forward(self, x):
    return self.network(x)

In [41]:
in_sz = X_train_tensor.shape[1]
model = ClassificationNN(in_sz)
model

ClassificationNN(
  (network): Sequential(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [42]:
# loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [43]:
# Train loop
epochs = 1000
for e in range(epochs):
  model.train()
  optimizer.zero_grad()

  output = model(X_train_tensor)
  loss = loss_function(output, y_train_tensor)
  loss.backward()
  optimizer.step()

  if (e + 1) % 100 == 0:
    print(f'Epoch [{e + 1}/{epochs}], loss = {loss.item():.4f}')

Epoch [100/1000], loss = 0.0291
Epoch [200/1000], loss = 0.0146
Epoch [300/1000], loss = 0.0029
Epoch [400/1000], loss = 0.0009
Epoch [500/1000], loss = 0.0004
Epoch [600/1000], loss = 0.0003
Epoch [700/1000], loss = 0.0002
Epoch [800/1000], loss = 0.0001
Epoch [900/1000], loss = 0.0001
Epoch [1000/1000], loss = 0.0001


In [53]:
# Evalidates the model
model.eval()
with torch.no_grad():
  y_predict = model(X_test_tensor)
  y_max_pred = y_predict.argmax(dim=1) # give the max index of tensor

  acc = y_test_tensor[y_test_tensor == y_max_pred].sum().item() / y_test_tensor.shape[0]
  print(f'Model Accuracy: {acc:.3f}')

Model Accuracy: 0.967


In [71]:
# Inference function
def one_prediction(sepal_length, sepal_width, petal_length, petal_width):
  pred_df = pd.DataFrame(data=[[sepal_length, sepal_width, petal_length, petal_width]], columns=iris_data.feature_names)

  pred_df = scaler.transform(pred_df)
  pred_df_tensor = torch.tensor(pred_df, dtype=torch.float32)

  predictions = model(pred_df_tensor)
  predictions_index = predictions.argmax(dim=1)
  predictions_label = iris_data.target_names[predictions_index]
  return predictions_label

In [72]:
labels = one_prediction(5.1, 3.5, 1.4, 0.2)
print('Label : ', labels)

Label :  setosa
